In [53]:
import pandas as pd
import numpy as np 
import pulp as p


# Gathering Data

In [54]:
skaters = pd.read_csv(r'D:\arbunize\skaters.csv')
goalies = pd.read_csv(r'D:\arbunize\goalies.csv')

In [55]:
skaters.head()

,Player Name,Likes,Inj,Pos,Salary,Team,Opp,Rest,Line,PP,...,S SOG,L5 FP,L40 FP,S FP,Floor FP,Ceil FP,Proj FP,Proj Val,Actual FP,Actual Val
0,Patrick Kane,NaN,NaN,RW,7500,CHI,DET,2.0,L3,P1,...,3.7,18.4,15.8,15.8,4.6,27.1,13.3,1.8,5.0,0.7
1,Evgeni Malkin,NaN,NaN,C,7200,PIT,FLA,0.0,L1,P1,...,3.5,18.0,15.5,15.5,5.2,25.8,11.5,1.6,13.1,1.8
2,Sidney Crosby,NaN,Q,C,7100,PIT,FLA,0.0,NaN,NaN,...,2.7,11.4,11.3,11.3,2.4,20.3,0.0,0.0,0.0,0.0
3,Nikita Kucherov,NaN,NaN,RW,7000,TB,@CAR,0.0,L1,P1,...,2.8,7.0,12.4,12.4,3.0,21.8,13.7,2.0,12.6,1.8
4,Roman Josi,NaN,NaN,D,7000,NSH,@ANH,0.0,D1,P1,...,3.6,20.0,16.1,16.1,7.8,24.5,10.8,1.5,10.6,1.5


In [56]:
goalies.head()

,Player Name,Likes,Inj,Pos,Salary,Team,Opp,Rest,Vegas Odds Win,Vegas Odds GA,...,S SVPCT,L5 FP,L40 FP,S FP,Floor FP,Ceil FP,Proj FP,Proj Val,Actual FP,Actual Val
0,Corey Crawford,NaN,NaN,G,8100,CHI,DET,2,NaN,NaN,...,0.905,9.3,12.3,12.3,1.5,23.1,16.2,2.0,12.3,1.5
1,Tristan Jarry,NaN,NaN,G,8000,PIT,FLA,0,NaN,NaN,...,0.936,18.1,17.9,17.9,8.2,27.5,15.0,1.9,11.2,1.4
2,Juuse Saros,NaN,NaN,G,7900,NSH,@ANH,0,NaN,NaN,...,0.890,6.1,8.0,8.0,-1.5,17.4,14.1,1.8,21.1,2.7
3,Petr Mrazek,NaN,NaN,G,7800,CAR,TB,1,NaN,NaN,...,0.901,7.9,11.3,11.3,1.3,21.2,11.0,1.4,5.6,0.7
4,Alex Stalock,NaN,NaN,G,7800,MIN,CGY,0,NaN,NaN,...,0.901,7.2,11.0,11.0,1.1,20.9,12.7,1.6,11.1,1.4


In [57]:
skaters.shape

(290, 26)

In [58]:
num_skaters = skaters.shape[0]

In [59]:
num_goalies = goalies.shape[0]

# Separating Skaters into 3 Categories

In [60]:
wingers =[]
centers =[]
defenders =[]
for i in range(num_skaters):
        
    if skaters['Pos'][i] == 'LW' or skaters['Pos'][i] == 'RW' or skaters['Pos'][i] == 'W':

        wingers.append(1)
        centers.append(0)
        defenders.append(0)
    elif skaters['Pos'][i] == 'C':

        wingers.append(0)
        centers.append(1)
        defenders.append(0)

    elif skaters['Pos'][i] == 'D' or skaters['Pos'][i] == 'LD' or skaters['Pos'][i] == 'RD':

        wingers.append(0)
        centers.append(0)
        defenders.append(1)

    else:

        wingers.append(0)
        centers.append(0)
        defenders.append(1)

    

In [61]:
len(wingers)

290

In [62]:
teams = list(np.unique(np.array(skaters['Team'])) )
    
num_teams = len(teams)

In [63]:
teams

['ANH', 'CAR', 'CGY', 'CHI', 'CLS', 'DET', 'FLA', 'MIN', 'NSH', 'PIT', 'TB']

# Mapping Each Skater with their Team

In [64]:
skaters_teams =[]
for i in range(num_skaters):
    info =[0]*num_teams
    for j in range(num_teams):
        if skaters['Team'][i] == teams[j]:
            info[j]=1
    skaters_teams.append(info)
            
    
    
    

In [65]:
len(skaters_teams)

290

# Mapping Skaters to the lines they belong to

In [66]:
team_lines=[]
for i in range(num_teams):

    l1 = [0]*num_skaters
    l2=l1
    l3=l1
    l4=l1
    lines=[]
    for j in range(num_skaters):

        if skaters['Team'][j] == teams[i]:

            if skaters['Line'][j]== 'L1':
                l1[j]=1
            elif skaters['Line'][j] == 'L2':
                l2[j]=1
            elif skaters['Line'][j] == 'L3':
                l3[j]=1
            elif skaters['Line'][j] == 'L4':
                l4[j]=1

    #lines.append(l1)
    #lines.append(l2)
    #lines.append(l3)
    #lines.append(l4)

    team_lines.append(l1)
    team_lines.append(l2)
    team_lines.append(l3)
    team_lines.append(l4)



# Mapping Skaters who play in First Powerplay

In [67]:
P1_info=[]
    
for i in range(num_teams):

    PP =[0]*num_skaters

    for j in range(num_skaters):
        if skaters['Team'][j]== teams[i]:
            if skaters['PP'][j]=='P1':
                PP[j]=1
    P1_info.append(PP)

# Performing Integer Programming to find the optimal lineup

###  ( doesn't consist of all the constraints )



In [68]:
prob = p.LpProblem('Problem',p.LpMaximize)


 # Variable for skaters in lineup
skaters_lineup = p.LpVariable.dict('skaterline',range(num_skaters),cat='Binary')
# Variable for goalie in lineup
goalies_lineup = p.LpVariable.dict('goaliesline',range(num_goalies),cat='Binary')

#Objective
prob+= sum([skaters['Proj FP'][i]*skaters_lineup[i] for i in range(num_skaters)] ) + sum([goalies['Proj FP'][i]*goalies_lineup[i] for i in range(num_goalies)] )

#Constraints

 # One goalie constraint
prob+= sum([goalies_lineup[i] for i in range(num_goalies)])==1

 # Eight skaters constraint
prob+=sum([skaters_lineup[i] for i in range(num_skaters)])==8

# between 2 and 3 centers
prob+=sum([centers[i]*skaters_lineup[i] for i in range(num_skaters)])<=3


prob+=sum([centers[i]*skaters_lineup[i] for i in range(num_skaters)])>=2

# between 3 and 4 wingers

prob+=sum([wingers[i]*skaters_lineup[i] for i in range(num_skaters)])<=4

prob+=sum([wingers[i]*skaters_lineup[i] for i in range(num_skaters)])>=3

# between 2 and 3 defenders
prob+=sum([defenders[i]*skaters_lineup[i] for i in range(num_skaters)])<=3

prob+=sum([defenders[i]*skaters_lineup[i] for i in range(num_skaters)])>=2

 # Financial Constraint
prob+=sum([skaters['Salary'][i]*skaters_lineup[i] for i in range(num_skaters)]) + sum([goalies['Salary'][i]*goalies_lineup[i] for i in range(num_goalies)]) <=50000 

#solving
status = prob.solve()
        


print(p.LpStatus[status])


skater_lineup_final =[]
goalie_lineup_final =[]
if p.LpStatus[status]=='Optimal':
    for i in range(num_skaters):
        if p.value(skaters_lineup[i])>=0.9 and p.value(skaters_lineup[i])<=1.1:
            skater_lineup_final.append(1)

        else:
            skater_lineup_final.append(0)


    for i in range(num_goalies):
        if p.value(goalies_lineup[i])>=0.9 and p.value(goalies_lineup[i])<=1.1:
            goalie_lineup_final.append(1)

        else:
            goalie_lineup_final.append(0)


for j in [skater_lineup_final[i]*skaters['Player Name'][i] for i in range(num_skaters)] :
    if j!='':
        print(j)
for j in [goalie_lineup_final[i]*goalies['Player Name'][i] for i in range(num_goalies)] :
    if j!='':
        print(j)

#print(prob)

Optimal
Nikita Kucherov
Dylan Larkin
Johnny Gaudreau
Aleksander Barkov
Viktor Arvidsson
Vincent Trocheck
Mikhail Sergachev
Mike Matheson
Corey Crawford


In [69]:
num_overlap = 7

In [70]:
num_lineups = 5
num_lines = len(team_lines)

# Performing Integer Programming to find Optimal Lineup 
###  (consists of all the constraints )


In [71]:
prob = p.LpProblem('Problem',p.LpMaximize)

#Skater Variable
skaters_lineup = p.LpVariable.dict('skaterline',range(num_skaters),cat='Binary')
#Goalies Variable
goalies_lineup = p.LpVariable.dict('goaliesline',range(num_goalies),cat='Binary')
# Used Teams Variable
used_teams = p.LpVariable.dict('usedteams',range(num_teams),cat='Binary')
# Line Stack Variable
line_stack = p.LpVariable.dict('linestack',range(num_lines),cat='Binary')







lin =[0]*(num_skaters+num_goalies)
lineups=[]
for i in range(num_lineups):
    lineups.append(lin)


#Objective
prob+= sum([skaters['Proj FP'][i]*skaters_lineup[i] for i in range(num_skaters)] ) + sum([goalies['Proj FP'][i]*goalies_lineup[i] for i in range(num_goalies)] )

#Constraints

 # One goalie constraint
prob+= sum([goalies_lineup[i] for i in range(num_goalies)])==1

 # Eight skaters constraint
prob+=sum([skaters_lineup[i] for i in range(num_skaters)])==8

# between 2 and 3 centers
prob+=sum([centers[i]*skaters_lineup[i] for i in range(num_skaters)])<=3


prob+=sum([centers[i]*skaters_lineup[i] for i in range(num_skaters)])>=2

# between 3 and 4 wingers

prob+=sum([wingers[i]*skaters_lineup[i] for i in range(num_skaters)])<=4

prob+=sum([wingers[i]*skaters_lineup[i] for i in range(num_skaters)])>=3

# between 2 and 3 defenders
prob+=sum([defenders[i]*skaters_lineup[i] for i in range(num_skaters)])<=3

prob+=sum([defenders[i]*skaters_lineup[i] for i in range(num_skaters)])>=2

 # Financial Constraint
prob+=sum([skaters['Salary'][i]*skaters_lineup[i] for i in range(num_skaters)]) + sum([goalies['Salary'][i]*goalies_lineup[i] for i in range(num_goalies)]) <=50000 


#Overlapping Constraint
for i in range(num_lineups):
    prob+= sum([lineups[i][j]*skaters_lineup[j] for j in range(num_skaters)] ) + sum([lineups[i][j+num_skaters]*goalies_lineup[j] for j in range(num_goalies)]) <=num_overlap

    
# Team Constarint should select from more than 3 teams    
for i in range(num_teams):
    prob+=sum([skaters_teams[j][i]*skaters_lineup[j] for j in range(num_skaters) ]) >=used_teams[i]

    
prob+= sum([used_teams[i] for i in range(num_teams)]) >=3


 # Must have at least one complete line in each lineup
for i in range(num_lines):
    
    prob+= sum([ team_lines[i][j]*skaters_lineup[j] for j in range(num_skaters) ]) >= 3*line_stack[i]
    

    
 # The defenders must be on Power Play 1
prob+= sum( [ sum ( [ defenders[i]*P1_info[j][i]*skaters_lineup[i] for i in range(num_skaters)    ] )  for j in range(num_teams)]  ) == sum( [defenders[i]*skaters_lineup[i] for i in range(num_skaters) ])      



#solving
status = prob.solve()
        


print(p.LpStatus[status])

skater_lineup_final =[]
goalie_lineup_final =[]
if p.LpStatus[status]=='Optimal':
    for i in range(num_skaters):
        if p.value(skaters_lineup[i])>=0.9 and p.value(skaters_lineup[i])<=1.1:
            skater_lineup_final.append(1)

        else:
            skater_lineup_final.append(0)


    for i in range(num_goalies):
        if p.value(goalies_lineup[i])>=0.9 and p.value(goalies_lineup[i])<=1.1:
            goalie_lineup_final.append(1)

        else:
            goalie_lineup_final.append(0)


for j in [skater_lineup_final[i]*skaters['Player Name'][i] for i in range(num_skaters)] :
    if j!='':
        print(j)
for j in [goalie_lineup_final[i]*goalies['Player Name'][i] for i in range(num_goalies)] :
    if j!='':
        print(j)

#print(prob)

Optimal
Nikita Kucherov
Aleksander Barkov
Sebastian Aho
Kris Letang
Viktor Arvidsson
Vincent Trocheck
Keith Yandle
Yanni Gourde
Corey Crawford
